<a href="https://colab.research.google.com/github/michelmalki/MichelMalki/blob/main/CCCS_680_Assignment_3_Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [199]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col, count

spark= SparkSession \
       .builder \
       .appName("CCCS 680 Assignment 3 Question 3") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [131]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [200]:
access_log_df = spark.read.csv('/content/drive/My Drive/CCCS-680 Assignment 3/access_log.txt.gz', inferSchema=True, header=False, sep='\t')

In [124]:
access_log_df.show()

+--------------------+
|                 _c0|
+--------------------+
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
|10.223.157.186 - ...|
+--------------------+
only showing top 20 rows



## 3.A
Display the number of hits for each different file on the web site

In [196]:
hits_df = access_log_df.groupBy(access_log_df._c0).count().orderBy(col('count'))
hits_df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+-----+
|_c0                                                                                                                       |count|
+--------------------------------------------------------------------------------------------------------------------------+-----+
|10.223.157.186 - - [15/Jul/2009:15:50:35 -0700] "GET /assets/js/the-associates.js HTTP/1.1" 200 4492                      |1    |
|10.223.157.186 - - [15/Jul/2009:20:50:33 -0700] "GET /assets/img/home-logo.png HTTP/1.1" 304 -                            |1    |
|10.223.157.186 - - [15/Jul/2009:20:50:39 -0700] "GET /favicon.ico HTTP/1.1" 404 209                                       |1    |
|10.82.30.199 - - [16/Jul/2009:03:00:29 -0700] "GET /trailers/ HTTP/1.1" 200 11508                                         |1    |
|10.82.30.199 - - [16/Jul/2009:03:00:06 -0700] "GET /assets/css/the-associates.css 

# 3.B
Determine the number of hits to the site made by each different IP address

In [201]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col

hits_df = access_log_df.groupBy('_c0').count().orderBy(col('count').desc())

print("Hits from each IP address:")
hits_df.show(truncate=False)
unique_ip_count = hits_df.count()
spark.stop()

Hits from each IP address:
+----------------------------------------------------------------------------------------------------------------------+-----+
|_c0                                                                                                                   |count|
+----------------------------------------------------------------------------------------------------------------------+-----+
|10.247.111.104 - - [07/Jan/2010:04:27:51 -0800] "GET /assets/img/banner/ten-years-banner-black.jpg HTTP/1.1" 200 16846|10   |
|10.221.179.195 - - [03/Jan/2010:08:34:14 -0800] "GET /robots.txt HTTP/1.0" 404 208                                    |6    |
|10.214.246.177 - - [07/Jan/2010:20:25:23 -0800] "GET /robots.txt HTTP/1.0" 404 208                                    |5    |
|10.196.218.47 - - [13/Jan/2010:09:27:03 -0800] "GET /robots.txt HTTP/1.0" 404 208                                     |5    |
|10.121.241.75 - - [10/Jan/2010:19:48:19 -0800] "GET /robots.txt HTTP/1.0" 404 208  

# 3.C
Find the most popular file on the web site. That is, the file whose path occurs
most often in access_log.

In [197]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, regexp_extract, col


hits_df = access_log_df.groupBy('_c0').count().orderBy(col('count').desc())
most_popular_file = hits_df.first()
N = 5
print(f"\nTop {N} most popular files:")
hits_df.show(N, False)
spark.stop()


Top 5 most popular files:
+----------------------------------------------------------------------------------------------------------------------+-----+
|_c0                                                                                                                   |count|
+----------------------------------------------------------------------------------------------------------------------+-----+
|10.247.111.104 - - [07/Jan/2010:04:27:51 -0800] "GET /assets/img/banner/ten-years-banner-black.jpg HTTP/1.1" 200 16846|10   |
|10.221.179.195 - - [03/Jan/2010:08:34:14 -0800] "GET /robots.txt HTTP/1.0" 404 208                                    |6    |
|10.214.246.177 - - [07/Jan/2010:20:25:23 -0800] "GET /robots.txt HTTP/1.0" 404 208                                    |5    |
|10.196.218.47 - - [13/Jan/2010:09:27:03 -0800] "GET /robots.txt HTTP/1.0" 404 208                                     |5    |
|10.121.241.75 - - [10/Jan/2010:19:48:19 -0800] "GET /robots.txt HTTP/1.0" 404 208  